# Python Geospatial Tools Webinar

## Foundational tools

The geospatial ecosystem in Python builds on a number of foundational tools much like the rest of the Python scientific ecosystem.

### Numpy

NumPy is the fundamental package for scientific computing in Python. It is a Python library that provides a multidimensional array object, various derived objects (such as masked arrays and matrices), and an assortment of routines for fast operations on arrays, including mathematical, logical, shape manipulation, sorting, selecting, I/O, discrete Fourier transforms, basic linear algebra, basic statistical operations, random simulation and much more.

- [Documentation](https://numpy.org/doc/stable/index.html)
- [GitHub](https://github.com/numpy/numpy)

Installation:

```
conda install numpy
```

In [ ]:
import numpy as np

a = np.array([1, 2, 3])

a

### Pandas

pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

- [Documentation](https://pandas.pydata.org/docs/)
- [GitHub](https://github.com/pandas-dev/pandas)

Installation:

```
conda install pandas
```

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "Name": [
            "Braund, Mr. Owen Harris",
            "Allen, Mr. William Henry",
            "Bonnell, Miss. Elizabeth",
        ],
        "Age": [22, 35, 58],
        "Sex": ["male", "male", "female"],
    }
)

df

### Dask

Dask is a flexible library for parallel computing in Python.

- [Documentation](https://docs.dask.org/en/latest/)
- [GitHub](https://github.com/pandas-dev/pandas)

Installation:

```
conda install dask
```

#### dask.distributed

Starting the Dask Client is optional. It will provide a dashboard which is useful to gain insight on the computation.

In [ ]:
from dask.distributed import Client, progress

client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='16GB')

client

#### dask.array

Dask arrays coordinate many Numpy arrays, arranged into chunks within a grid. They support a large subset of the Numpy API.

In [ ]:
import dask.array as da

x = da.random.random((10000, 10000), chunks=(1000, 1000))

x

In [ ]:
summed = x.sum(axis=0)

summed

In [ ]:
summed.compute()

#### dask.dataframe

Dask Dataframes coordinate many Pandas dataframes, partitioned along an index. They support a large subset of the Pandas API.

In [ ]:
import dask

import dask.dataframe as dd

ddf = dask.datasets.timeseries()

ddf

In [ ]:
ddf2 = ddf[ddf.y > 0]

ddf3 = ddf2.groupby('name').x.std()

ddf3.compute()

### Numba

Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

- [Documentation](https://numba.pydata.org/)
- [GitHub](https://github.com/numba/numba)

Installation:

```
conda install numba
```

In [ ]:
from numba import jit

x = np.arange(100).reshape(10, 10)

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

print(go_fast(x))

### Cupy

CuPy is an implementation of NumPy-compatible multi-dimensional array on CUDA. CuPy consists of cupy.ndarray, the core multi-dimensional array class, and many functions on it. It supports a subset of numpy.ndarray interface.

- [Documentation](https://docs.cupy.dev/en/stable/)
- [GitHub](https://github.com/cupy/cupy)

Installation:

```
conda install -c conda-forge cupy
```

In [ ]:
import cupy as cp

x = cp.arange(6).reshape(2, 3).astype('f')

x

In [ ]:
x.sum(axis=1)

## Vector Data

<img src="vector_data2.png"></img>

### Geopandas

GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and matplotlib for plotting.

- [Documentation](https://geopandas.org/docs.html)
- [GitHub](https://github.com/geopandas/geopandas)

Installation:

```
conda install -c conda-forge geopandas
```

In [ ]:
import geopandas as gpd

url = "http://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_land.geojson"
gdf = gpd.read_file(url)

gdf

Indexing the geometry column returns `shapely` objects:

In [ ]:
gdf.geometry.iloc[21]

However underlying this is a `GeometryArray` class:

In [ ]:
type(gdf.geometry.values)

which stores pointers to GEOS objects:

In [ ]:
gdf.geometry.values.data[:1]

### spatialpandas

Spatialpandas provides Pandas and Dask extensions for vectorized spatial and geometric operations, such as fast, spatially indexed rendering of large collections of polygons, lines, or points.

- [Documentation](https://nbviewer.jupyter.org/github/holoviz/spatialpandas/blob/master/examples/Overview.ipynb)
- [GitHub](https://github.com/holoviz/spatialpandas)

Installation:

```
conda install -c holoviz spatialpandas
```

spatialpandas does not provide any file access or I/O support for classic geospatial formats but does allow storing geometry data in parquet files. It also transparently converts Geopandas objects:

In [ ]:
import spatialpandas as spd

sdf = spd.GeoDataFrame(gdf)

sdf

Instead of storing shapely or GEOS objects, spatialpandas declares geometry array classes:

In [ ]:
type(sdf.geometry.values)

which stores the geometry data as a contiguous PyArrow `ListArray`:

In [ ]:
sdf.geometry.values.data[:1]

#### Spatial indexing

Spatialpandas supports spatially indexing a GeoDataFrame of geometries ensuring that nearby geometries are stored in the same partitions. This ensures that without loading data in memory we can quickly look up specific geometries with a spatial query.

In [ ]:
sidf = dd.from_pandas(sdf, npartitions=8)

sidf

In [ ]:
import datashader as ds

def plot_partitions(ddf):
    # Get divisions array
    divs = np.array(ddf.divisions)[:-1]
    
    # Add categorical "partition" column
    ddf2 = ddf.map_partitions(
        lambda df: df.assign(
            partition=pd.Categorical(np.searchsorted(divs, df.index, side="right"))
        )
    ).compute()
    
    # Create Datashader image, coloring countries by partition
    cvs = ds.Canvas(plot_width=650, plot_height=400)
    agg = cvs.polygons(ddf2, geometry='geometry', agg=ds.count_cat('partition'))
    return ds.transfer_functions.shade(agg)

plot_partitions(sidf)

In [ ]:
sidf.cx[-80:-85]

#### Spatial Joins

In [ ]:
minx, miny, maxx, maxy = sidf.geometry.total_bounds

N = 1_000_000

df_points = gpd.GeoDataFrame(
    {"id": np.arange(N)},
    geometry=gpd.points_from_xy(minx + maxx * np.random.random(N), miny + maxy * np.random.random(N)),
    crs=gdf.crs
)

sdf_points = dd.from_pandas(spd.GeoDataFrame(df_points), npartitions=16).persist()

In [ ]:
%%time
spd.sjoin(sdf_points, sdf).compute()

### dask-geopandas

Parallel GeoPandas with Dask supporting some of the same features as the DaskGeoDataFrame such as spatial indexing and spatial joins.

- [Documentation](https://nbviewer.jupyter.org/github/holoviz/spatialpandas/blob/master/examples/Overview.ipynb)
- [GitHub](https://github.com/geopandas/dask-geopandas/)

Installation:

```
pip install dask-geopandas
```

In [ ]:
import dask_geopandas

In [ ]:
ddf = dask_geopandas.from_geopandas(df_points, npartitions=4)

In [ ]:
%%time
dask_geopandas.sjoin(ddf, gdf).compute()

### cuspatial

- [Documentation](https://intake.readthedocs.io/en/latest/)
- [GitHub](https://github.com/rapidsai/cuspatial)

Installation:

```
conda install -c conda-forge -c rapidsai-nightly cuspatial
```

## Raster data

To represent and work with raster data in Python there are a number of essential tools which in turn build on some or all of the foundational tools above.

<img src="gridded_types.png"></img>

### xarray

Xarray is an open source project and Python package that provides a toolkit for working with labeled multi-dimensional arrays of data. Xarray adopts the Common Data Model for self-describing scientific data in widespread use in the Earth sciences: `xarray.Dataset` is an in-memory representation of a netCDF file. Xarray provides the basic data structures, as well as powerful tools for computation and visualization.

- [Documentation](https://xarray.pydata.org/en/stable/)
- [GitHub](https://github.com/pydata/xarray)

Installation:

```
conda install xarray
```

#### Loading data

In [ ]:
import xarray as xr

air_temp = xr.tutorial.open_dataset('air_temperature')

air_temp

In [ ]:
air_temp.sel({'lat': slice(75, 50), 'lon': slice(200, 220)})

### Datashader

*Accurately render even the largest data*

Datashader is a graphics pipeline system for creating meaningful representations of large datasets quickly and flexibly. Datashader breaks the creation of images into a series of explicit steps that allow computations to be done on intermediate representations. This approach allows accurate and effective visualizations to be produced automatically without trial-and-error parameter tuning, and also makes it simple for data scientists to focus on particular data and relationships of interest in a principled way.

The computation-intensive steps in this process are written in ordinary Python but transparently compiled to machine code using Numba and flexibly distributed across CPU cores and processors using Dask or GPUs using CUDA. This approach provides a highly optimized rendering pipeline that makes it practical to work with extremely large datasets even on standard hardware, while exploiting distributed and GPU systems when available.

- [Documentation](https://datashader.org/)
- [GitHub](https://github.com/holoviz/datashader)

Installation:

```
conda install datashader
```

<style type="text/css">.arbit .trary a { color: inherit; }.arbit .trary
.sL{text-align:center;padding:2px 2px 2px 2px;background-color:#ffffff;font-weight:bold;width:60px}.arbit .trary
.sG{text-align:center;padding:2px 2px 2px 2px;background-color:#ffffff;font-weight:bold;font-family:monospace}.arbit .trary
.sY{text-align:center;padding:2px 2px 2px 2px;background-color:#b7e1cd;}.arbit .trary 
.sN{text-align:center;padding:2px 2px 2px 2px;background-color:#f4c7c3;}.arbit .trary
.sM{text-align:center;padding:2px 2px 2px 2px;background-color:#fce8b2;}.arbit .trary
</style>
<div class="arbit">
<table class="trary" cellspacing="0" cellpadding="0">
<thead>
<tr>
  <th class="sG">Data object</th>
  <th class="sG">Structure</th>
  <th class="sG">Compute</th>
  <th class="sG">Memory</th>
  <th class="sG">Description</th>
  <th class="sG">points</th>
  <th class="sG">line</th>
  <th class="sG">area</th>
  <th class="sG">trimesh</th>
  <th class="sG">raster</th>
  <th class="sG">quadmesh</th>
  <th class="sG">polygons</th>
</tr>
</thead>
<tbody>
<tr>
  <td class="sL"><a href="https://pandas.pydata.org">Pandas DF</a></td>
  <td>columnar</td>
  <td>1-core CPU</td>
  <td>in-core</td>
  <td>Standard dataframes</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
</tr>
<tr>
  <td class="sL"><a href="https://dask.org">DaskDF + PandasDF</a></td>
  <td>columnar</td>
  <td>distributed CPU</td>
  <td>out-of-core</td>
  <td>Distributed DataFrames</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
</tr>
<tr>
  <td class="sL"><a href="https://github.com/rapidsai/cudf">cuDF</a></td>
  <td>columnar</td>
  <td>single GPU</td>
  <td>in-core</td>
  <td>NVIDIA GPU DataFrames</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sN">No</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
</tr>
<tr>
  <td class="sL"><a href="https://docs.rapids.ai/api/cudf/stable/10min.html">DaskDF + cuDF</a></td>
  <td>columnar</td>
  <td>distributed GPU</td>
  <td>out-of-core</td>
  <td>Distributed NVIDIA GPUs</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sN">No</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
</tr>
<tr>
  <td class="sL"><a href="https://github.com/holoviz/spatialpandas">SpatialPandasDF</a></td>
  <td>ragged</td>
  <td>1-core CPU</td>
  <td>in-core</td>
  <td>Ragged + spatial indexing</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sY">Yes</td>
</tr>
<tr>
  <td class="sL"><a href="https://github.com/holoviz/spatialpandas">Dask + SpatialPandasDF</a></td>
  <td>ragged</td>
  <td>distributed CPU</td>
  <td>out-of-core</td>
  <td>Distributed ragged arrays</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sM">-</td>
  <td class="sY">Yes</td>
</tr>
<tr>
  <td class="sL"><a href="http://xarray.pydata.org">Xarray + NumPy</a></td>
  <td>n-D</td>
  <td>1-core CPU</td>
  <td>in-core</td>
  <td>n-D CPU arrays</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sM">-</td>
</tr>
<tr>
  <td class="sL"><a href="(https://dask.org">Xarray+DaskArray</a></td>
  <td>n-D</td>
  <td>distributed CPU</td>
  <td>out-of-core</td>
  <td>Distributed n-D arrays</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sY">Yes</td>
  <td class="sY">Yes</td>
  <td class="sM">-</td>
</tr>
<tr>
  <td class="sL"><a href="https://cupy.chainer.org">Xarray+CuPy</a></td>
  <td>n-D</td>
  <td>single GPU</td>
  <td>in-core</td>
  <td>Single-GPU n-D arrays</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sN">No</td>
  <td class="sY">Yes</td>
  <td class="sM">-</td>
</tr>
</tbody></table></div>


In [ ]:
import datashader as ds

W = 1280
H = 768

#### Points

In [ ]:
points_df = dd.read_parquet('/Users/philippjfr/development/datashader/examples/data/nyc_taxi.parq/').compute()

points_df

In [ ]:
%%time
cvs = ds.Canvas(plot_width=W, plot_height=H)

pickup_agg = cvs.points(points_df, 'pickup_x', 'pickup_y')

pickup_agg

In [ ]:
ds.tf.shade(pickup_agg, how='log')

#### Line

In [ ]:
streets_df = spd.GeoDataFrame(gpd.read_file('./streets.json'))

streets_df.head()

In [ ]:
%%time
cvs = ds.Canvas(plot_width=850, plot_height=500)

street_agg = cvs.line(streets_df, geometry='geometry')

street_agg

In [ ]:
ds.tf.shade(street_agg)

#### Polygons

In [ ]:
poly_df = spd.GeoDataFrame(pd.read_parquet('./nyc_buildings.parq/'))

poly_df['type'] = poly_df['type'].astype('category')

In [ ]:
len(poly_df)

In [ ]:
%%time
cvs = ds.Canvas(plot_width=W, plot_height=H)

poly_agg = cvs.polygons(poly_df, geometry='geometry', agg=ds.reductions.count_cat('type'))

poly_agg

In [ ]:
import colorcet

ds.tf.shade(poly_agg, color_key=colorcet.glasbey)

## xarray-spatial

Xarray-Spatial implements common raster analysis functions using Numba and provides an easy-to-install, easy-to-extend codebase for raster analysis.

xarray-spatial does not depend on GDAL / GEOS, which makes it fully extensible in Python but does limit the breadth of operations that can be covered. xarray-spatial is meant to include the core raster-analysis functions needed for GIS developers / analysts, implemented independently of the non-Python geo stack.

- [Documentation](https://xarray-spatial.org/)
- [GitHub](https://github.com/makepath/xarray-spatial)

Installation:

```
conda install -c conda-forge xarray-spatial
```

In [ ]:
from xrspatial import generate_terrain

W = 800
H = 600

cvs = ds.Canvas(plot_width=W, plot_height=H, x_range=(-20e6, 20e6), y_range=(-20e6, 20e6))

terrain = generate_terrain(canvas=cvs)

ds.tf.shade(terrain, cmap=['black', 'white'], how='linear')

In [ ]:
from xrspatial import hillshade
from xrspatial import quantile

qcut_agg = quantile(terrain, k=15)

ds.tf.stack(
    ds.tf.shade(hillshade(qcut_agg), cmap=['gray', 'white'], alpha=255, how='linear'),
    ds.tf.shade(qcut_agg,     cmap=ds.colors.Elevation,      alpha=128, how='linear')
)

In [ ]:
most_common = list(poly_df.type.value_counts().sort_values().iloc[-50:].index)

filtered_polys = poly_df[poly_df.type.isin(most_common)].copy()

filtered_polys['zones'] = [most_common.index(t) for t in filtered_polys.type]

cvs = ds.Canvas(
    plot_width=5000, plot_height=5000,
    x_range=(pickup_agg.pickup_x.min().item(), pickup_agg.pickup_x.max().item()),
    y_range=(pickup_agg.pickup_y.min().item(), pickup_agg.pickup_y.max().item())
)

zones_agg = cvs.polygons(filtered_polys, geometry='geometry', agg=ds.reductions.min('zones'))

pickup_agg = cvs.points(points_df, 'pickup_x', 'pickup_y', agg=ds.sum('passenger_count'))
dropoff_agg = cvs.points(points_df, 'dropoff_x', 'dropoff_y', agg=ds.sum('passenger_count'))

In [ ]:
import colorcet as cc

ds.tf.stack(
    ds.tf.shade(zones_agg, cmap=cc.glasbey),
    ds.tf.shade(pickup_agg, cmap=['white', 'red'])
)

In [ ]:
from xrspatial import zonal_stats

custom_stats = dict(passenger_pickups=np.sum)

pickup_stats = zonal_stats(zones_agg, pickup_agg, zone_ids=list(range(50)), stats_funcs=custom_stats)
pickup_stats.insert(0, 'Type', [most_common[z] for z in pickup_stats.zone])
pickup_stats.sort_values('passenger_pickups', ascending=False).iloc[:20]

In [ ]:
custom_stats = dict(passenger_dropoffs=np.sum)

dropoff_stats = zonal_stats(zones_agg, dropoff_agg, zone_ids=list(range(50)), stats_funcs=custom_stats)
dropoff_stats.insert(0, 'Type', [most_common[z] for z in dropoff_stats.zone])
dropoff_stats.sort_values('passenger_dropoffs', ascending=False).iloc[:20]

## Data catalogs

### Intake

Intake is a lightweight set of tools for loading and sharing data in data science projects.

- [Documentation](https://intake.readthedocs.io/en/latest/)
- [GitHub](https://github.com/intake/intake)

Installation:

```
conda install intake
```

Example catalog:

```yaml
plugins:
  source:
    - module: intake_xarray
sources:
  esgf:
    description: CREATE sample
    driver: opendap
    args:
      urlpath: 'http://esgf.nccs.nasa.gov/thredds/dodsC/CREATE-IP/reanalysis/ECMWF/IFS-Cy31r2/day/atmos/pr/pr_day_reanalysis_IFS-Cy31r2_19790101-19791231.nc'
      chunks: {}
      xarray_kwargs:
        decode_times: False
        
  geotiff:
    description: Geotiff image of Landsat Surface Reflectance Level-2 Science Product L5.
    driver: rasterio
    cache:
      - argkey: urlpath
        regex: 'earth-data/landsat'
        type: file
    args:
      urlpath: 's3://earth-data/landsat/small/LT05_L1TP_042033_{collection_date:%Y%m%d}_{processing_date:%Y%m%d}_01_T1_sr_band{band:1d}.tif'
      chunks:
        band: 1
        x: 50
        y: 50
      concat_dim: band
      storage_options: {'anon': True}
    metadata:
      plots:
        band_image:
          kind: 'image'
          x: 'x'
          y: 'y'
          groupby: 'band'
          rasterize: True
```

In [ ]:
import intake

cat = intake.open_catalog('/Users/philippjfr/development/hvplot/examples/datasets.yaml')

cat.us_crime.to_dask()

### STAC


- [Documentation](https://pystac.readthedocs.io/en/1.0/#)
- [GitHub](https://github.com/stac-utils/pystac)

Installation:

```
conda install intake
```

## Visualization

### Matplotlib

Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python.

- [Documentation](https://matplotlib.org/stable/contents.html)
- [GitHub](https://github.com/matplotlib/matplotlib)

Installation:

```
conda install matplotlib
```

#### Geopandas

In [ ]:
%matplotlib inline

gdf.plot()

#### Xarray

In [ ]:
air_temp.air.isel(time=0).plot()

#### Cartopy

Cartopy is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses.
    
- [Documentation](https://scitools.org.uk/cartopy/docs/latest/)
- [GitHub](https://github.com/SciTools/cartopy)

Installation:

```
conda install cartopy
```

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

# make the map global rather than have it zoom in to
# the extents of any plotted data
ax.set_global()

ax.stock_img()
ax.coastlines()

ax.plot(-0.08, 51.53, 'o', transform=ccrs.PlateCarree())
ax.plot([-0.08, 132], [51.53, 43.17], transform=ccrs.PlateCarree())
ax.plot([-0.08, 132], [51.53, 43.17], transform=ccrs.Geodetic())

### hvPlot

hvPlot provides a high-level plotting API built on HoloViews that provides a general and consistent API for plotting data originating from Pandas, Dask DataFrames, cuDF, xarray, GeoPandas and Spatialpandas. hvPlot can integrate neatly with the individual libraries if an extension mechanism for the native plot APIs is offered, or it can be used as a standalone component.

- [Documentation](https://hvplot.holoviz.org/)
- [GitHub](https://github.com/holoviz/hvplot)

Installation:

```
conda install hvplot
```

In [ ]:
import hvplot.xarray
import hvplot.pandas

#### Datashader integration

In [ ]:
points_df.hvplot.points('pickup_x', 'pickup_y', rasterize=True, cnorm='log')

In [ ]:
import colorcet as cc

poly_df.hvplot.polygons(datashade=True, aggregator=ds.count_cat('type'), color_key=cc.glasbey)

#### Geopandas support

In [ ]:
gdf.hvplot()

#### xarray support

In [ ]:
air_temp.hvplot.image()

#### Cartopy projection support via GeoViews

In [ ]:
proj = ccrs.Orthographic(-90, 30)

air_temp.air.hvplot.quadmesh(
    'lon', 'lat', projection=proj, project=True, global_extent=True, 
    cmap='viridis', rasterize=True, coastline=True, frame_width=500)

### ipyleaflet

Interactive maps in the Jupyter notebook

- [Documentation](https://ipyleaflet.readthedocs.io/en/latest/index.html)
- [GitHub](https://github.com/jupyter-widgets/ipyleaflet)

Installation:

```
conda install -c conda-forge ipyleaflet
```

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    basemap=basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"),
    center=(52.204793, 360.121558),
    zoom=4
)

m

### pydeck

High-scale spatial rendering in Python, powered by deck.gl.

- [Documentation](https://deckgl.readthedocs.io/en/latest/index.html)
- [GitHub](https://github.com/visgl/deck.gl/tree/master/bindings/pydeck)

Installation:

```
conda install -c conda-forge pydeck
```

In [ ]:
import pydeck as pdk

DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

INITIAL_VIEW_STATE = pdk.ViewState(latitude=49.254, longitude=-123.13, zoom=11, max_zoom=16, pitch=45, bearing=0)

polygon = pdk.Layer(
    "PolygonLayer",
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 20],
)

geojson = pdk.Layer(
    "GeoJsonLayer",
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation="properties.valuePerSqm / 20",
    get_fill_color="[255, 255, properties.growth * 255]",
    get_line_color=[255, 255, 255],
)

pdk.Deck(layers=[polygon, geojson], initial_view_state=INITIAL_VIEW_STATE)